# Import Library

This code is for importing library what I need.

In [1]:
!pip install numpy
!pip install pandas==1.5.3
!pip install nltk
!pip install beautifulsoup4
!pip install mpld3
!pip install -U scikit-learn
!pip install joblib
!pip install gensim
!pip install pyLDAvis==3.4.0

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from bs4 import BeautifulSoup
import re
import os 
import codecs
from sklearn import feature_extraction
import mpld3
from glob import glob 
import json

import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-05 07:36:39.547774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 07:36:39.731991: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudar

# Document Clustering

### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the titles/bodies. First, I load NLTK's list of English stop words. Stop words are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [3]:
# load nltk's English stopwords as variable called 'stopwords'
new_stopwords = ["use", "abl", "would", "like", "made", "happen", "id", "n\'t", "expect", "1", "useline", "arn", "aw", "int2byt", "py35", "python", "kw", "def", "six", "six.mov", "\'s", "ns", "-r", "int2byte_alt", "python3", "six.add_metaclass", "setup.pi", "import_modul", "r", "x", "__name__", "0m", "1m" ,"\'m", "'m", "flask", "pkgutil.get_load", "ani", "-g", "_configtest.c", "unknown._", "f", "\'d", "|sin", "|cos", "h", "numpy.float64", "|squar", "|absolut", "|sqrt", "|reciproc", "b", "-fwrapv", "defin", "x86_64-linux-gnu-gcc", "-fstack-protector-strong", "numpi", "µs", "ms", "//projects.scipy.org/numpy/ticket/709", "v", "datetime.datetime.now", "side=\'left", "bench_avx", "v_shape", "np.searchsort", "__init__.pi", "array_lik", "ss", "cdk", "aws-cdk", "kwarg", "name", "iam", "aws-cdk/cor", "aws-cdk/cfnspec", "callabl", "arg", "l", "0x485b9e", "0x0", "cjw296", "numpy.isnan", "xxx", "\\n", "__init__"]
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(new_stopwords)
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Next I import the Snowball Stemmer which is actually part of NLTK. Stemming is just the process of breaking a word down into its root.

In [4]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

Below I define two functions: 
tokenize_and_stem: tokenizes (splits the titles into a list of its respective words (or tokens) and also stems each token tokenize_only: tokenizes the titles only I use both these functions to create a dictionary which becomes important in case I want to use stems for an algorithm, but later convert stems back to their full words for presentation purposes. Guess what, I do want to do that!

In [5]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

### Latent Dirichlet Allocation

For my implementaiton of LDA, I use the Gensim package. I'm going to preprocess the titles a bit differently here, and first I define a function to remove any proper noun.

In [6]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [7]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [8]:
#Latent Dirichlet Allocation implementation with Gensim

from gensim import corpora, models, similarities 

## Title and Body Newcomers Scenario

### Stopwords, stemming, and tokenizing

Access, split and compute data from newcomers body and title txt files.

In [9]:
#import data title
titles = open('data_title_expert_newcomers_combined.txt').read().split('\BREAKHERE')

#ensures that only the first 1825 are read in
titles = titles[1:13633]

#import data body
bodies = open('data_body_expert_newcomers_combined.txt').read().split('\BREAKHERE')

#ensures that only the first 1825 are read in
bodies = bodies[1:13633]

print(str(len(titles)) + ' titles')
print(str(len(bodies)) + ' bodies')

135 titles
135 bodies


Enter newcomers body and title data into the list

In [10]:
# generates index for each item
list_titles = []

for text in titles:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    list_titles.append(text)

titles = list_titles

# generates index for each item
list_bodies = []

for text in bodies:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    list_bodies.append(text)

bodies = list_bodies

bodies_titles = []

for i in range(len(bodies)):
    item = bodies[i] + titles[i]
    bodies_titles.append(item)

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Check newcomers body and title data.

In [11]:
pd.DataFrame(bodies_titles)

,0
0,Originally reported by: **Eklavya Sharma (Bitb...
1,Originally reported by: **Ernest Prabhakar (Bi...
2,Originally reported by: **petrcermak (Bitbucke...
3,Originally reported by: **philbeaudoin (Bitbuc...
4,Originally reported by: **Wouter Bolsterlee (B...
...,...
130,Originally reported by: **Zearin (Bitbucket: [...
131,Originally reported by: **Michael Salib (Bitbu...
132,Originally reported by: **Matt Chaput (Bitbuck...
133,Originally reported by: **Sridhar Ratnakumar (...


Below I use my stemming/tokenizing and tokenizing functions to iterate over the list of bodies and titles to create two vocabularies: one stemmed and one only tokenized.

In [12]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in bodies_titles:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

Using these two lists, I create a pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column. The benefit of this is it provides an efficient way to look up a stem and return a full token. The downside here is that stems to tokens are one to many: the stem 'run' could be associated with 'ran', 'runs', 'running', etc. For my purposes this is fine--I'm perfectly happy returning the first token associated with the stem I need to look up.

In [13]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

Check newcomers bodies and titles data vocab frame.

In [14]:
vocab_frame

,words
origin,originally
report,reported
by,by
eklavya,eklavya
sharma,sharma
...,...
path,path
bitbucket,bitbucket
https,https
//bitbucket.org/gutworth/six/issue/1,//bitbucket.org/gutworth/six/issue/1


Check the shape of items in vocab_frame

In [15]:
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 11455 items in vocab_frame


### Latent Dirichlet Allocation

Here I run the actual text processing (removing of proper nouns, tokenization, removal of stop words)

In [16]:
#Latent Dirichlet Allocation implementation with Gensim

#remove proper names
preprocess = [strip_proppers(doc) for doc in bodies_titles]

%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

CPU times: user 187 ms, sys: 0 ns, total: 187 ms
Wall time: 190 ms
CPU times: user 34.9 ms, sys: 0 ns, total: 34.9 ms
Wall time: 34.9 ms


In [17]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

22


Below are some Gensim specific conversions; I also filter out extreme words (see inline comment)

In [18]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [19]:
len(corpus)

135

The actual model runs below. I took 1825 passes to ensure convergence, but you can see that it took my machine 4 minutes to run. My chunksize is larger than the corpus so basically all synopses are used per pass. I should optimize this, and Gensim has the capacity to run in parallel. I'll likely explore this further as I use the implementation on larger corpora.

In [20]:
# %time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, update_every=5, chunksize=10000, passes=100)
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

CPU times: user 8.6 s, sys: 0 ns, total: 8.6 s
Wall time: 8.71 s


Each topic has a set of words that defines it, along with a certain probability.

In [21]:
# lda.show_topics()

pprint(lda.print_topics())

[(0,
  '0.026*"line" + 0.015*"modul" + 0.014*"class" + 0.013*"move" + '
  '0.011*"return" + 0.010*"base" + 0.009*"import" + 0.009*"function" + '
  '0.008*"metaclass" + 0.008*"doe"'),
 (1,
  '0.017*"return" + 0.017*"iter" + 0.017*"import" + 0.011*"iterkey" + '
  '0.010*"support" + 0.009*"run" + 0.008*"http" + 0.008*"renam" + '
  '0.007*"itervalu" + 0.007*"peterjc"'),
 (2,
  '0.035*"line" + 0.028*"import" + 0.019*"loop" + 0.011*"return" + 0.009*"per" '
  '+ 0.009*"best" + 0.009*"timeit" + 0.009*"modul" + 0.008*"pass" + '
  '0.008*"class"'),
 (3,
  '0.019*"except" + 0.013*"traceback" + 0.011*"modul" + 0.011*"doe" + '
  '0.010*"version" + 0.010*"support" + 0.010*"import" + 0.007*"rais" + '
  '0.007*"tri" + 0.007*"differ"'),
 (4,
  '0.013*"import" + 0.012*"modul" + 0.011*"return" + 0.008*"function" + '
  '0.007*"method" + 0.006*"differ" + 0.006*"version" + 0.006*"func" + '
  '0.006*"provid" + 0.006*"line"')]


In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

/root/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.130367 -0.037614       1        1  28.619575
0      0.085309  0.030426       2        1  22.994797
4     -0.089492 -0.072903       3        1  19.931635
1     -0.059159  0.146461       4        1  18.203013
3     -0.067025 -0.066369       5        1  10.250980, topic_info=            Term       Freq      Total Category  logprob  loglift
117         loop  29.000000  29.000000  Default  30.0000  30.0000
290         line  93.000000  93.000000  Default  29.0000  29.0000
7         except  17.000000  17.000000  Default  28.0000  28.0000
148         iter  18.000000  18.000000  Default  27.0000  27.0000
150      iterkey  10.000000  10.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
272       provid   2.596983  16.169050   Topic5  -5.3064   0.4490
154         nice   1.962588  11.741327   Topic5  -5.5865   0.4889
380       becaus   1.961574   9.467608   Topic5  -5.5870   0.7037
1121  six.rerais   1.960555   5.570803   Topic5  -5.5875   1.2335
21        compat   1.960382  11.789444   Topic5  -5.5876   0.4837

[299 rows x 6 columns], token_table=      Topic      Freq                                   Term
term                                                        
393       5  0.509915                              -c'import
206       1  0.754831                                     -n
207       1  0.754831                                    -r5
234       5  0.509915  //bitbucket.org/gutworth/six/issue/78
235       5  0.509915                   //bitbucket.org/nehz
...     ...       ...                                    ...
15        3  0.155762                                   work
15        4  0.389405                                   work
1602      4  0.681941                                  write
440       5  0.509915                                  wrong
162       4  0.818533                                  xflr6

[444 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 5, 2, 4])

In [23]:
pyLDAvis.save_html(vis, 'LDA_Combined.html')

In [24]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.31100973637907114


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a884c52-566d-41cd-8e39-3e39f7a04cb7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>